In [14]:
import numpy as np
import math

In [15]:
# state space:
# coordinates relative to checkpoints coordinates C = (x, y)
# velocity relative to C
# acceleration relative to C
# angular velocity relativ to global coordinate system
# angualar acceleration relativ to global coordinate system
# difference vector between optimal direction of drone body and current direction (normalised unit vectors)

class DroneEnv:
    def __init__(self, width=1000, height=1000, checkpoint_radius=60):
        # environment
        self.width = width
        self.height = height
        self.checkpoint_radius = checkpoint_radius

        # environment params:
        self.GRAVITY = 2.5
        self.THRUST_POWER = 5
        self.X_AXIS_SENS = 50
        self.Y_AXIS_SENS = 1
        self.ROTATION_SPEED = 1
        self.ROTATION_DRAG = 0.7
        self.MOVEMENT_DRAG = 0.5

        # initiate drone
        self.reset(x=self.width / 2, y=self.height / 2, vx=0, vy=0, ax=0, ay=0, a=0, va=0)

    def reset(self, x, y, vx, vy, ax, ay, a, va):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.ax = ax
        self.ay = ay
        self.a = a # in degrees
        self.va = va

        self.spawn_checkpoint()

        return self.get_state() # not likely to be of use

    def spawn_checkpoint(self):
        self.checkpoint_x = np.random.uniform(0.1 * self.width, 0.9 * self.width)
        self.checkpoint_y = np.random.uniform(0.1 * self.height, 0.9 * self.height)

    def step(self, action):
        # logic: ax,ay -> change vx,vy -> change x, y ; va -> changes angle

        thrust_left = action[0] # thrust values
        thrust_right = action[1]

        # physics:
        self.vy += self.GRAVITY # Y

        torque = thrust_left - thrust_right # factor

        self.va -= self.ROTATION_SPEED * torque # VA
        thrust = self.THRUST_POWER * math.cos(math.radians(self.a)) * ((thrust_left + thrust_right) / 2)

        self.vy -= self.Y_AXIS_SENS * thrust # Y

        self.va *= self.ROTATION_DRAG

        self.a += self.va
        self.a %= 360.0

        angle_rad = math.radians(self.a)
        self.vx = self.X_AXIS_SENS * math.sin(angle_rad)

        self.x += self.MOVEMENT_DRAG * self.vx
        self.y += self.MOVEMENT_DRAG * self.vy

        dx = self.checkpoint_x - self.x
        dy = self.checkpoint_y - self.y
        dist = math.sqrt(dx * dx + dy * dy)

        reached = False
        reward = -dist * 0.01

        if dist < self.checkpoint_radius:
            reached = True
            reward += 10.0
            self.spawn_checkpoint()

        out = False
        if self.x < 0 or self.x > self.width or self.y < 0 or self.y > self.height:
            out = True
            reward -= 5.0

        return self.get_state(), reward, out, reached

    def get_state(self):
        pass


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = torch.device("cuda")
torch.cuda.is_available()

#print(torch.version.cuda)


ModuleNotFoundError: No module named 'torch'

In [17]:
# visualisation for debugging:
import pygame
import math
import numpy as np
from IPython.display import display, clear_output # because this is .ipynb

# drone
env = DroneEnv(width=1000, height=1000)
env.reset(x=env.width / 2, y=env.height / 2, vx=0, vy=0, ax=0, ay=0, a=0, va=0)

# pygame env
pygame.init()
screen = pygame.display.set_mode((env.width, env.height))
pygame.display.set_caption("DroneEnv Visualisation")
clock = pygame.time.Clock()

# constants
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED   = (255, 0, 0)
GREEN = (0, 255, 0)
DRONE_W = 60
DRONE_H = 20

# drwaing function for pygame
def draw_drone(screen, x, y, angle_deg):
    angle = math.radians(angle_deg)
    hw, hh = DRONE_W / 2, DRONE_H / 2

    points = [(-hw, -hh), ( hw, -hh), ( hw,  hh), (-hw,  hh)] # vertices of drone (just a block)

    rot = [] # derive rotated vertices
    for px, py in points:
        rx = px * math.cos(angle) - py * math.sin(angle)
        ry = px * math.sin(angle) + py * math.cos(angle)
        rot.append((x + rx, y + ry)) # coordinates + relative vertex coordinates = absolute coordinates

    pygame.draw.polygon(screen, WHITE, rot) # pygame screen, colour, absolute drone vertices

# pygame variables:
running = True
pause = False

while running:
    # event handling for pygame
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed() # user input / alternativ zu KI-inputs

    # thrust control
    thrust_left  = 0.70 if keys[pygame.K_LEFT]  else 0.0
    thrust_right = 1.0 if keys[pygame.K_RIGHT] else 0.0

    if keys[pygame.K_ESCAPE]:
        running = False

    if keys[pygame.K_p]:
        pause = not pause
        pygame.time.wait(200) # prevent spamming keys

    if not pause:
        _, reward, out, reached = env.step([thrust_left, thrust_right]) # update state via physics

        if out: # when outside of box
            env.reset(x=env.width / 2, y=env.height / 2, vx=0, vy=0, ax=0, ay=0, a=0, va=0)

    # update screen:
    screen.fill(BLACK)

    # checkpoint
    pygame.draw.circle(
        screen,
        GREEN,
        (int(env.checkpoint_x), int(env.checkpoint_y)),
        env.checkpoint_radius,
        2
    )

    draw_drone(screen, env.x, env.y, env.a) # drone with new state

    pygame.display.flip()
    clock.tick(60)
pygame.quit()


In [18]:
# Training FFN policy:




